read single cell object data

In [ ]:
import json
from livecell_tracker.core.single_cell import SingleCellTrajectoryCollection, SingleCellStatic
from livecell_tracker.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)

all_scs_json_path = "./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json"
all_scs = SingleCellStatic.load_single_cells_json(all_scs_json_path)
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

read trained an MMAction model


In [2]:
import torch

path = r"D:\LiveCellTracker-dev\notebooks\scripts\mmdetection_classify\work_dirs\tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v3\epoch_4758.pth"
model = torch.load(path)

In [3]:
from mmengine.config import Config, DictAction
from mmaction.registry import MODELS
config_file = r"D:\LiveCellTracker-dev\notebooks\scripts\mmdetection_classify\tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
checkpoint_file = r"D:\LiveCellTracker-dev\notebooks\scripts\mmdetection_classify\work_dirs\tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v3\epoch_4758.pth"
cfg = Config.fromfile(config_file)

In [4]:
# model = MODELS.build()
import mmcv
from mmaction.apis import init_recognizer, inference_recognizer
model = init_recognizer(config_file, checkpoint_file, device='cuda:0')

Loads checkpoint by local backend from path: D:\LiveCellTracker-dev\notebooks\scripts\mmdetection_classify\work_dirs\tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb-v3\epoch_4758.pth


In [5]:
from mmcv.video.io import VideoReader
import tempfile
import numpy as np

In [6]:
from livecell_tracker.track.classify_utils import video_frames_and_masks_from_sample, combine_video_frames_and_masks

In [7]:
sample = sctc.get_all_trajectories()[10].get_all_scs()

print("length of sample:", len(sample))
frames, frame_masks = video_frames_and_masks_from_sample(sample)


length of sample: 32


In [8]:
combined_frames = combine_video_frames_and_masks(frames, frame_masks)
sct_input_frames = [np.array(frame) for frame in combined_frames]

In [9]:
import cv2
import os
import uuid
# video = r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data\videos\mitosis_1_mask.mp4"
# video = r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data\videos\normal_6.mp4"

tmp_id = uuid.uuid4()
fps = 1
tmp_dir = "./tmp_video"
os.makedirs(tmp_dir, exist_ok=True)
filename = os.path.join(tmp_dir, f"tmp-{tmp_id}.mp4")
input_frames = sct_input_frames[20:30]
with open(filename, "wb+") as tmp_video_file:
    # mmcv.frames2video(frames, tmp_video_file.name, fps=1)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # Create a VideoWriter object to write the frames to the output file
    height, width, channels = input_frames[0].shape
    video_writer = cv2.VideoWriter(tmp_video_file.name, fourcc, fps, (width, height))

    # Write the frames to the output file
    for frame in input_frames:
        video_writer.write(frame)
    video_writer.release()
    tmp_video_file.flush()
    
results = inference_recognizer(model, tmp_video_file.name)
print(tmp_video_file)
results

06/24 13:15:11 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
06/24 13:15:11 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
<_io.BufferedRandom name='./tmp_video\\tmp-b9619f08-a50d-45d6-9938-75a79552fb91.mp4'>


<ActionDataSample(

    META INFORMATION
    img_shape: (224, 224)

    DATA FIELDS
    pred_scores: <LabelData(
        
            META INFORMATION
        
            DATA FIELDS
            item: tensor([2.5799e-06, 9.7249e-09, 1.0000e+00], device='cuda:0')
        ) at 0x1d367add640>
    pred_labels: <LabelData(
        
            META INFORMATION
        
            DATA FIELDS
            item: tensor([2], device='cuda:0')
        ) at 0x1d367add5e0>
    gt_labels: <LabelData(
        
            META INFORMATION
        
            DATA FIELDS
            item: tensor([-1], device='cuda:0')
        ) at 0x1d367add550>
) at 0x1d367add5b0>

In [13]:
results = inference_recognizer(model, r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data_v3\videos\normal_0_combined_padding-20.mp4")
results

<ActionDataSample(

    META INFORMATION
    img_shape: (224, 224)

    DATA FIELDS
    pred_scores: <LabelData(
        
            META INFORMATION
        
            DATA FIELDS
            item: tensor([1.4891e-04, 1.3151e-06, 9.9985e-01], device='cuda:0')
        ) at 0x1d368e15640>
    pred_labels: <LabelData(
        
            META INFORMATION
        
            DATA FIELDS
            item: tensor([2], device='cuda:0')
        ) at 0x1d368e15730>
    gt_labels: <LabelData(
        
            META INFORMATION
        
            DATA FIELDS
            item: tensor([-1], device='cuda:0')
        ) at 0x1d368e15b50>
) at 0x1d368e157f0>